# Covid-19 en Centroamérica
> Dashboard

- toc: false
- branch: master
- badges: true
- comments: false
- image: images/covid-19-ca.png
- permalink: /covid-19-ca/
- author: "Libre AI"
- categories: [centroamerica, covid19]

In [17]:
#hide
import math
import folium
import pandas as pd
import flag

from IPython.display import display, display_html, HTML, IFrame, Markdown

In [18]:
#hide_input

from datetime import datetime
utc_now = datetime.utcnow()

Markdown('> Last updated on {} UTC'.format(utc_now.strftime("%Y-%m-%d %H:%M")))

> Last updated on 2020-03-31 12:25 UTC

In [19]:
#hide

#latest case count
country_code2name = {'gt': 'Guatemala',
                     'bz': 'Belize',
                     'hn': 'Honduras',
                     'sv': 'El Salvador',
                     'ni': 'Nicaragua',
                     'cr': 'Costa Rica',
                     'pa': 'Panama'
                    }



In [20]:
#hide
confirmed = pd.read_csv('data/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('data/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('data/time_series_covid19_recovered_global.csv')

In [21]:
#hide
def get_latest_count(df, country):
    df_country = df[df['Country/Region'] == country]
    # dates are columns, so we take the last column
    latest_count_date = df_country.columns[-1]
    latest_count = df_country[latest_count_date]
    return latest_count.values[0]

In [22]:
#hide 
latest_cases_by_country = {c_code:get_latest_count(confirmed, country_code2name[c_code]) for c_code in country_code2name}

latest_deaths_by_country = {c_code:get_latest_count(deaths, country_code2name[c_code]) for c_code in country_code2name}

latest_recovered_by_country = {c_code:get_latest_count(recovered, country_code2name[c_code]) for c_code in country_code2name}

('latest_deaths_by_country', latest_deaths_by_country,
 'latest_deaths_by_country', latest_deaths_by_country,
 'latest_recovered_by_country', latest_recovered_by_country)

('latest_deaths_by_country',
 {'gt': 1, 'bz': 0, 'hn': 7, 'sv': 0, 'ni': 1, 'cr': 2, 'pa': 24},
 'latest_deaths_by_country',
 {'gt': 1, 'bz': 0, 'hn': 7, 'sv': 0, 'ni': 1, 'cr': 2, 'pa': 24},
 'latest_recovered_by_country',
 {'gt': 10, 'bz': 0, 'hn': 3, 'sv': 0, 'ni': 0, 'cr': 4, 'pa': 4})

In [23]:
#hide
#The Map ---

In [24]:
#hide

map_center_lat, map_center_long = (12.5, -87)
zoom = 6

In [25]:
#hide

class Country:
    def __init__(self, name, lat, long):
        super()
        self.cases = 0
        self.deaths = 0
        self.recovered = 0
        self.name = name
        self.lat = lat
        self.long = long


In [26]:
#hide

gt = Country(country_code2name['gt'], 14.63, -90.56)
bz = Country(country_code2name['bz'], 17.25, -88.80)
hn = Country(country_code2name['hn'], 14.08, -87.24)
sv = Country(country_code2name['sv'], 13.69, -89.25)
ni = Country(country_code2name['ni'], 12.10, -86.33)
cr = Country(country_code2name['cr'], 9.94, -84.15)
pa = Country(country_code2name['pa'], 9.08, -79.59)

countries = {'gt': gt, 'bz': bz, 'hn': hn, 'sv': sv, 'ni': ni, 'cr': cr, 'pa': pa}

In [27]:
#hide

# set latest cases
for c in countries:
    countries[c].cases = latest_cases_by_country[c]
    countries[c].deaths = latest_deaths_by_country[c]
    countries[c].recovered = latest_recovered_by_country[c]
    

In [28]:
#hide

m = folium.Map(location=[map_center_lat, map_center_long],
               zoom_start=zoom,
               max_zoom=zoom+1,
               min_zoom=zoom-1
              )

# tiles = 'cartodbdark_matter'
# tiles = 'cartodbpositron'
# tiles = 'stamentoner'
tiles = 'stamenterrain'

folium.TileLayer(tiles, min_zoom=zoom-1, max_zoom=zoom+1).add_to(m)

tooltip_template = '''<div>
<b>{}</b><br/>
<span style='color:orange'><h3 style='display:inline'>{}</h3></span>&nbsp;Diagnosticados<br/>
<span style='color:red'><h4 style='display:inline'>{}</h4></span>&nbsp;Muertos<br/>
<span style='color:green'><h4 style='display:inline'>{}</h4></span>&nbsp;Curados
</div>
'''

for c in countries:
    folium.CircleMarker(
        location=[countries[c].lat, countries[c].long],
        radius=math.sqrt(countries[c].cases),
        color='red',
        fill=True,
        fill_color='#f03',
        tooltip=tooltip_template.format(countries[c].name,
                                        countries[c].cases,
                                        countries[c].deaths,
                                        countries[c].recovered
                                       )
    ).add_to(m)


m.save('../maps/covid19-ca-map.html')

In [29]:
# hide_input
IFrame('../maps/covid19-ca-map.html', width=800, height=600)

---

In [30]:
#hide
# The table ---

In [31]:
#hide

def get_summary_table():
    country_code_name = sorted(country_code2name.items(), key=lambda x: x[1])
    country_code = [x[0] for x in country_code_name]
    country_name = ["{} {}".format(x[1], flag.flag(x[0])) for x in country_code_name]

    cases_per_country = []
    deaths_per_country = []
    recovered_per_country = []

    for c in country_code:
        cases_per_country.append(latest_cases_by_country[c])
        deaths_per_country.append(latest_deaths_by_country[c])
        recovered_per_country.append(latest_recovered_by_country[c])
    
    x = pd.DataFrame({'País': country_name,
                      'Diagnosticados': cases_per_country,
                      'Muertos': deaths_per_country,
                      'Curados': recovered_per_country
                     })
    return x



In [32]:
#hide_input

summary_df = get_summary_table()

heading_properties = [('font-size', '22px')]
cell_properties = [('font-size', '18px')]
dfstyle = [dict(selector="th", props=heading_properties), 
           dict(selector="td", props=cell_properties)]

df_style = summary_df.style\
.hide_index()\
.background_gradient(cmap='Oranges', subset=['Diagnosticados'])\
.background_gradient(cmap='Reds', subset=['Muertos'])\
.background_gradient(cmap='Greens', subset=['Curados'])\
.set_table_styles(dfstyle)

display(HTML('<div style="display: flex; justify-content: center;">{}</div>'.format(df_style.render())))



País,Diagnosticados,Muertos,Curados
Belize 🇧🇿,3,0,0
Costa Rica 🇨🇷,330,2,4
El Salvador 🇸🇻,30,0,0
Guatemala 🇬🇹,36,1,10
Honduras 🇭🇳,139,7,3
Nicaragua 🇳🇮,4,1,0
Panama 🇵🇦,989,24,4


---
### Data Sources
- 2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE [https://github.com/CSSEGISandData/COVID-19]
